In [4]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
data = pd.read_excel('RS_KSR_ACT_WFM_STAT.xlsx')
pd.set_option('display.max_rows', 100)

In [6]:
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

In [7]:
data = data[~data .offered.isna()]

In [18]:
import pandas as pd

In [ ]:
pd.

In [8]:
data = data[data .offered != 0]

In [17]:
data = data[data.datetime<=pd.to_datetime('2025-04-01 00:00:00')]

In [10]:
# 假设数据有两列：'timestamp' 和 'value'
holiday_data = pd.read_excel(r'holiday.xlsx')
holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date

data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')

# 获取所有唯一的时间点（如每天的09:00, 09:30等）
unique_times = data['time'].unique()

# 获取所有唯一的日期
unique_dates = data['date'].unique()
data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

/tmp/ipykernel_1389/1831622517.py:12: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')


In [11]:
holiday_datetime = holiday_data.date.to_numpy()

In [12]:
from datetime import time

In [13]:
unique_times = [ time(8, 0), time(8, 30),
time(9, 0), time(9, 30), time(10, 0),
time(10, 30), time(11, 0), time(11, 30),
time(12, 0), time(12, 30), time(13, 0),
time(13, 30), time(14, 0), time(14, 30),
time(15, 0), time(15, 30), time(16, 0),
time(16, 30), time(17, 0), time(17, 30),
time(18, 0), time(18, 30), time(19, 0)]

In [11]:
start_time = 35+23 # 58 how many days in advance
end_time = start_time+36 #36 is the train length

In [ ]:
300 tian 
qian 200 
0-36 + di 37 + 35+23
1-37 + di 38
...
163-200 + di 201

164-201 + di 202
263-58 - 300-58 + i 300-58

205 - 231 yuc 301+58

In [12]:
add_column = ['actans','actabn','absActHt','absActSa']

In [13]:
add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]

In [18]:
matrix = []

# 遍历每个时间点
for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
    # 过滤出当前时间点的数据
    time_data = data[data['time'] == time].set_index('date')['offered']    
    full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
    full_dates = full_dates.difference(holiday_datetime)
    time_data = time_data.groupby(time_data.index).sum()
    time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    add_data = data[data['time'] == time].set_index('date')[add_column]
    add_data = add_data.groupby(add_data.index).sum()
    add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    time_data1 = data[data['time'] == time1].set_index('date')['offered']    
    time_data1 = time_data1.groupby(time_data1.index).sum()
    time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
    
    time_data2 = data[data['time'] == time2].set_index('date')['offered']    
    time_data2 = time_data2.groupby(time_data2.index).sum()
    time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    year_data = time_data.shift(251)
    

    # 将当前时间点、前半小时和后半小时的数据拼接
    combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
    
    # 创建日期+时间列
    datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
    
    # 将日期+时间列添加到数据中
    combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
    
    # 将结果存入矩阵
    matrix.append(combined_data_with_datetime)

max_rows = max(arr.shape[0] for arr in matrix)

# 将每个时间点的数据填充到最大行数
for i in range(len(matrix)):
    if matrix[i].shape[0] < max_rows:
        padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
        matrix[i] = np.vstack([matrix[i], padding])

# 将矩阵堆叠成一个大的二维数组
matrix = np.vstack(matrix)

matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                      + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                      [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                       ['years_data']).sort_values('datetime').reset_index(drop = True)

matrix['year'] = matrix.datetime.map(lambda x:x.year)
matrix['month'] = matrix.datetime.map(lambda x:x.month)
matrix['day'] = matrix.datetime.map(lambda x:x.day)
matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
matrix['minute'] = matrix.datetime.map(lambda x:x.minute)

matrix = matrix.sort_values('datetime').reset_index(drop = True)

read_data = matrix.iloc[:]
read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)

fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))


In [19]:

fill_na = fill_na.interpolate(method='linear')

fill_na['datetime'] = read_data.datetime
fill_na['date'] = fill_na.datetime.dt.date
fill_na = fill_na.dropna().reset_index(drop = True)

fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
fill_na['weekday'] = fill_na['datetime'].dt.day_name()
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

In [21]:
matrix_predict = []

# 遍历每个时间点
for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
    # 过滤出当前时间点的数据
    time_data = data[data['time'] == time].set_index('date')['offered']    
    full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
    full_dates = full_dates.difference(holiday_datetime)
    time_data = time_data.groupby(time_data.index).sum()    
    time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    add_data = data[data['time'] == time].set_index('date')[add_column]
    add_data = add_data.groupby(add_data.index).sum()
    add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    
    time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
    time_data1 = time_data1.groupby(time_data1.index).sum()
    time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
    
    time_data2 = data[data['time'] == time2].set_index('date')['offered']  
    time_data2 = time_data2.groupby(time_data2.index).sum()
    time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
    shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
    
    
    year_data = time_data.shift(251)
    

    # 将当前时间点、前半小时和后半小时的数据拼接
    combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
    
    # 创建日期+时间列
    datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
    
    # 将日期+时间列添加到数据中
    combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
    
    
    
    # 将结果存入矩阵
    matrix_predict.append(combined_data_with_datetime)

max_rows = max(arr.shape[0] for arr in matrix_predict)
# 将每个时间点的数据填充到最大行数
for i in range(len(matrix_predict)):
    if matrix_predict[i].shape[0] < max_rows:
        padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
        matrix_predict[i] = np.vstack([matrix_predict[i], padding])

# 将矩阵堆叠成一个大的二维数组
matrix_predict = np.vstack(matrix_predict)

matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                      + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                       ['years_data']).sort_values('datetime').reset_index(drop = True)

matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)

matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)

read_data_predict = matrix_predict.iloc[:]


In [22]:
fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))

fill_na_predict = fill_na_predict.interpolate(method='linear')
fill_na_predict = fill_na_predict.fillna(1)

fill_na_predict['datetime'] = read_data_predict.datetime
fill_na_predict['date'] = fill_na_predict.datetime.dt.date

fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)

In [23]:
fill_na_predict

,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,actans_68,...,week_of_year_23,week_of_year_24,week_of_year_25,week_of_year_26,week_of_year_27,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,0.0,1.0,1.000,1.0,0.0,2.0,1.000,1.0,0.0,5.0,...,0,0,0,0,0,1,0,0,0,0
1,2.0,3.0,1617.254,22.0,4.0,2.0,3514.003,231.0,8.0,5.0,...,0,0,0,0,0,1,0,0,0,0
2,91.0,3.0,76213.335,15731.0,103.0,15.0,100157.300,41007.0,145.0,33.0,...,0,0,0,0,0,1,0,0,0,0
3,70.0,3.0,72582.115,25197.0,83.0,23.0,66921.806,92615.0,89.0,73.0,...,0,0,0,0,0,1,0,0,0,0
4,70.0,6.0,58213.909,21813.0,73.0,17.0,71601.083,49133.0,129.0,28.0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1329,30.0,0.0,23473.051,271.0,41.0,0.0,50386.345,658.0,52.0,0.0,...,0,0,0,0,1,0,0,1,0,0
1330,35.0,0.0,36502.894,1795.0,33.0,4.0,36368.028,1399.0,30.0,0.0,...,0,0,0,0,1,0,0,1,0,0
1331,20.0,2.0,26534.375,3631.0,24.0,5.0,20344.240,6246.0,21.0,0.0,...,0,0,0,0,1,0,0,1,0,0
1332,15.0,2.0,19090.379,2956.0,19.0,2.0,19822.707,4207.0,19.0,0.0,...,0,0,0,0,1,0,0,1,0,0


In [24]:
X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
            [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                add_columns
            
X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
            [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
            [f'year_{i}' for i in np.unique(fill_na.year)] +\
            [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                add_columns


lstm_data = fill_na.dropna(axis = 0)

In [25]:
lstm_data

,offered,actans_59,actans_60,actans_61,actans_62,actans_63,actans_64,actans_65,actans_66,actans_67,...,week_of_year_48,week_of_year_49,week_of_year_50,week_of_year_51,week_of_year_52,weekday_Friday,weekday_Monday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
0,37.0,32.0,1.0,11708.178571,3425.931624,29.0,2.0,8473.533333,3326.211905,35.0,...,0,0,0,1,0,1,0,0,0,0
1,17.0,34.0,3.0,15849.500000,15690.933333,24.0,3.0,12873.785714,1832.750000,24.0,...,0,0,0,1,0,1,0,0,0,0
2,33.0,24.0,17.0,16142.923077,13223.523810,34.0,3.0,22268.708333,13466.242857,26.0,...,0,0,0,1,0,1,0,0,0,0
3,40.0,33.0,9.0,14344.684848,22917.783333,27.0,1.0,16926.976190,11750.277778,21.0,...,0,0,0,1,0,1,0,0,0,0
4,38.0,20.0,6.0,35237.400000,18675.350000,58.0,5.0,35575.240000,10236.400000,32.0,...,0,0,0,1,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12402,37.0,39.0,25.0,42107.372000,43170.000000,36.0,15.0,41410.083000,31063.000000,31.0,...,0,0,0,0,0,0,0,0,1,0
12403,32.0,36.0,20.0,41885.721000,55599.000000,44.0,19.0,57229.320000,49366.000000,38.0,...,0,0,0,0,0,0,0,0,1,0
12404,20.0,26.0,13.0,35606.907000,38265.000000,28.0,13.0,29468.592000,32670.000000,20.0,...,0,0,0,0,0,0,0,0,1,0
12405,21.0,25.0,19.0,34078.505000,36963.000000,20.0,13.0,21124.890000,23320.000000,16.0,...,0,0,0,0,0,0,0,0,1,0


In [26]:
for i in X_columns:
    try:
        fill_na_predict[i]
    except:
        fill_na_predict[i] = 0

In [27]:
lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)

In [28]:
week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}

In [29]:
lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])

In [30]:
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape


In [34]:
now_lstm_data

,offered,freq_59,freq_60,freq_61,freq_62,freq_63,freq_64,freq_65,freq_66,freq_67,...,absActSa_85,absActSa_86,absActSa_87,absActSa_88,absActSa_89,absActSa_90,absActSa_91,absActSa_92,absActSa_93,absActSa_94
0,2.0,18.5,2.000000,1.0,24.000000,27.5,11.0,33.333333,1.000000,19.0,...,25082.060556,13509.944444,18.333333,0.666667,12567.188552,1429.500000,23.000000,1.666667,18419.362454,4620.681746
1,2.0,2.0,4.500000,2.0,1.000000,13.5,9.0,7.000000,20.666667,1.0,...,8120.250000,291.000000,19.666667,1.000000,11353.013889,1770.366667,15.000000,0.000000,6421.037037,378.466667
2,1.0,7.5,6.500000,2.0,4.500000,2.0,1.0,13.500000,9.000000,7.0,...,14103.629630,1424.541667,16.000000,0.333333,9912.000000,575.888889,12.333333,1.666667,8120.250000,291.000000
3,1.0,2.0,27.666667,10.5,22.000000,2.0,11.5,2.000000,1.000000,14.0,...,13034.808730,1546.642484,22.333333,0.666667,20413.129630,1250.740741,17.000000,1.000000,16438.518519,805.597222
4,1.0,7.0,4.000000,2.0,18.333333,7.5,6.5,2.000000,4.500000,2.0,...,9104.780952,1356.222222,15.000000,0.000000,9337.391453,1344.972222,22.333333,2.000000,19754.142063,1629.975817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,4.0,4.0,2.000000,5.0,2.000000,2.0,1.0,2.000000,7.000000,2.0,...,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
126,2.0,2.0,5.000000,4.0,2.000000,5.0,2.0,2.000000,1.000000,2.0,...,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,2.000000,1.000000,1.000000
127,1.0,1.0,2.000000,5.0,4.000000,2.0,5.0,2.000000,2.000000,1.0,...,3591.949333,58.333333,0.000000,2.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000
128,1.0,10.5,1.000000,2.0,5.000000,4.0,2.0,5.000000,2.000000,2.0,...,216.247000,6.000000,5.000000,0.666667,3591.949333,58.333333,0.000000,2.000000,1.000000,1.000000


In [32]:
all_predict = pd.DataFrame()
all_feature = pd.DataFrame()

In [35]:

all_predict = pd.DataFrame()
all_feature = pd.DataFrame()
#for week_ in [1,2,3,4,5]:
for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
    now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
    #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]

    now_time_list = now_lstm_data.datetime.to_numpy()
    now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
    # 归一化数据

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
    # 设置时间步长
    time_step = start_time
    # 创建训练数据集
    X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data

    X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]

    X_train_1 = np.array(X_train_1)
    y_train = np.array(y_train)
    X_test_1 = np.array(X_test_1)
    y_test = np.array(y_test)
    X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

    # 重塑输入数据为 [samples, time steps, features] 格式
    X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
    # 构建LSTM模型
    model_1 = Sequential()
    model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model_1.add(LSTM(50, return_sequences=False))
    model_1.add(Dense(25))
    model_1.add(Dense(1))

    # 编译模型
    model_1.compile(optimizer='adam', loss='mean_squared_error')
    model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
    train_predict_1 = model_1.predict(X_train_1)
    train_predict_1 = scaler.inverse_transform(train_predict_1)
    True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
    print(calculate_mape(train_predict_1, True_data_1))
    return_y_1 =  model_1.predict(X_test_1)
    return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
    now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
    now_predict_1['datetime'] = now_time_list[-start_time:]


    now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
    # 设置时间步长
    time_step = start_time
    # 创建训练数据集

    X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data

    X_test_2, y_test =now_lstm_data[X_columns2][-start_time:] , now_lstm_data[['offered']][-start_time:]

    X_train_2 = np.array(X_train_2)
    y_train = np.array(y_train)
    X_test_2 = np.array(X_test_2)
    y_test = np.array(y_test)
    X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)

    # 重塑输入数据为 [samples, time steps, features] 格式
    X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
    # 构建LSTM模型
    model_2 = Sequential()
    model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
    model_2.add(LSTM(50, return_sequences=False))
    model_2.add(Dense(25))
    model_2.add(Dense(1))

    # 编译模型
    model_2.compile(optimizer='adam', loss='mean_squared_error')
    model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
    train_predict_2 = model_2.predict(X_train_2)
    train_predict_2 = scaler.inverse_transform(train_predict_2)
    True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
    print(calculate_mape(train_predict_2, True_data_2))
    return_y_2 =  model_2.predict(X_test_2)
    return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
    now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
    now_predict_2['datetime'] = now_time_list[-start_time:]
    print(calculate_mape(y_test, return_y_pred_2))
    if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
        all_predict = pd.concat([now_predict_1,all_predict])
        future_X = now_fill_na_predict[X_columns].to_numpy()
        future_data = scaler.inverse_transform(np.array(\
                        model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
        future_data = pd.DataFrame(future_data,columns = ['predict'])
        all_feature = pd.concat([all_feature,future_data])
    else:
        all_predict = pd.concat([now_predict_2,all_predict])
        future_X = now_fill_na_predict[X_columns2].to_numpy()
        future_data = scaler.inverse_transform(np.array(\
                        model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
        future_data = pd.DataFrame(future_data,columns = ['predict'])
        all_feature = pd.concat([all_feature,future_data])
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
all_feature.to_excel(r'return2.xlsx')

Epoch 1/20


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 151ms/step - loss: 0.0659
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.0453
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 0.0532
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.0463
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 158ms/step - loss: 0.0475
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0407
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0510
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0435
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0478
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0448
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.0463
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.0401
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 0.0454
Epoch 14/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 0.0449
Epoch 15/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0398
Epoch 16/20
3/3 ━━━━━━━━━━━━━━

C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 152ms/step - loss: 0.2893
Epoch 2/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.1029
Epoch 3/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 0.0961
Epoch 4/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 0.0485
Epoch 5/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0738
Epoch 6/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.0663
Epoch 7/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 149ms/step - loss: 0.0488
Epoch 8/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0479
Epoch 9/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.0506
Epoch 10/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 0.0447
Epoch 11/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 0.0406
Epoch 12/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step - loss: 0.0447
Epoch 13/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 155ms/step - loss: 0.0366
Epoch 14/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 0.0436
Epoch 15/20
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step - loss: 0.0494
Epoch 16/20
3/3 ━━━━━━━━━━━━━━

...

17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 167ms/step - loss: 0.0377
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0184
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0160
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0175
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - loss: 0.0151
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - loss: 0.0184
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - loss: 0.0141
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0152
Epoch 9/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0138
Epoch 10/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0160
Epoch 11/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0129
Epoch 12/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - loss: 0.0158
Epoch 13/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - loss: 0.0135
Epoch 14/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - loss: 0.0151
Epoch 15/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0152


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 165ms/step - loss: 0.0463
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0267
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0255
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 167ms/step - loss: 0.0227
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0221
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0239
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0241
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0229
Epoch 9/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0213
Epoch 10/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0232
Epoch 11/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 163ms/step - loss: 0.0206
Epoch 12/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - loss: 0.0239
Epoch 13/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - loss: 0.0206
Epoch 14/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 173ms/step - loss: 0.0202
Epoch 15/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 169ms/step - loss: 0.0236


C:\Users\nemo\anaconda3\envs\lstm_env\lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


17/17 ━━━━━━━━━━━━━━━━━━━━ 5s 164ms/step - loss: 0.0863
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0322
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 166ms/step - loss: 0.0254
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0224
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0231
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0238
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - loss: 0.0224
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0203
Epoch 9/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - loss: 0.0232
Epoch 10/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - loss: 0.0241
Epoch 11/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 168ms/step - loss: 0.0215
Epoch 12/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 164ms/step - loss: 0.0231
Epoch 13/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 161ms/step - loss: 0.0204
Epoch 14/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 162ms/step - loss: 0.0206
Epoch 15/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 3s 165ms/step - loss: 0.0216


In [ ]:
all_predict['diff'] = (all_predict.predict - all_predict.offered)

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [ ]:
pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [ ]:
calculate_mape(all_predict.offered,all_predict.predict)

In [36]:
all_predict.to_excel(r'return1-.xlsx')

In [40]:
data[add_column + ['offered']].corr()

,actans,actabn,absActHt,absActSa,offered
actans,1.000000,0.067562,0.891382,0.306075,0.881601
actabn,0.067562,1.000000,0.159377,0.632675,0.389850
absActHt,0.891382,0.159377,1.000000,0.400931,0.887095
absActSa,0.306075,0.632675,0.400931,1.000000,0.495375
offered,0.881601,0.389850,0.887095,0.495375,1.000000


In [47]:
all_feature

,predict
0,2.465809
1,3.281004
2,2.308758
3,2.679123
4,2.779515
...,...
53,13.024989
54,13.060239
55,13.140845
56,13.205191


In [41]:
all_predict = pd.concat([now_predict_1,all_predict])
future_X = now_fill_na_predict[X_columns].to_numpy()
future_data = scaler.inverse_transform(np.array(\
                model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
future_data = pd.DataFrame(future_data,columns = ['predict'])
all_feature = pd.concat([all_feature,future_data])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step


In [43]:
all_predict

,predict,datetime,offered
0,13.169405,2025-01-23 19:00:00,NaN
1,13.432196,2025-01-24 19:00:00,NaN
2,13.250965,2025-01-27 19:00:00,NaN
3,13.148877,2025-01-28 19:00:00,NaN
4,13.017861,2025-01-29 19:00:00,NaN
...,...,...,...
1329,44.976898,2025-04-15 17:00:00,37.0
1330,37.730900,2025-04-15 17:30:00,32.0
1331,33.240807,2025-04-15 18:00:00,20.0
1332,27.432621,2025-04-15 18:30:00,21.0


In [44]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [45]:
all_predict.sort_values('datetime')

,predict,datetime,offered_x,offered_y
58,2.115029,2025-01-02 08:00:00,2.0,2.0
59,2.106689,2025-01-03 08:00:00,7.0,7.0
60,2.005751,2025-01-06 08:00:00,2.0,2.0
61,1.855358,2025-01-07 08:00:00,1.0,1.0
62,1.850668,2025-01-08 08:00:00,2.0,2.0
...,...,...,...,...
1388,37.730900,2025-04-15 17:30:00,32.0,32.0
1389,33.240807,2025-04-15 18:00:00,20.0,20.0
1390,27.432621,2025-04-15 18:30:00,21.0,21.0
57,13.227732,2025-04-15 19:00:00,NaN,12.0


In [51]:
all_feature = all_feature[:-58]

In [53]:
all_feature

,predict
0,2.465809
1,3.281004
2,2.308758
3,2.679123
4,2.779515
...,...
53,14.935564
54,14.947142
55,14.834311
56,14.826351


In [54]:
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
all_feature.to_excel(r'feature2.xlsx')

C:\Users\nemo\AppData\Local\Temp\ipykernel_6016\1055264202.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_feature['datetime'] =  np.hstack(time_)
C:\Users\nemo\AppData\Local\Temp\ipykernel_6016\1055264202.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_feature.predict = all_feature.predict.map(lambda x:max(0,x))


In [56]:
all_predict

,predict,datetime,offered_x,offered_y
0,13.169405,2025-01-23 19:00:00,NaN,25.0
1,13.432196,2025-01-24 19:00:00,NaN,12.0
2,13.250965,2025-01-27 19:00:00,NaN,28.0
3,13.148877,2025-01-28 19:00:00,NaN,19.0
4,13.017861,2025-01-29 19:00:00,NaN,17.0
...,...,...,...,...
1387,44.976898,2025-04-15 17:00:00,37.0,37.0
1388,37.730900,2025-04-15 17:30:00,32.0,32.0
1389,33.240807,2025-04-15 18:00:00,20.0,20.0
1390,27.432621,2025-04-15 18:30:00,21.0,21.0


In [55]:
all_feature

,predict,datetime
0,2.465809,2025-04-11 08:00:00
1,3.281004,2025-04-14 08:00:00
2,2.308758,2025-04-15 08:00:00
3,2.679123,2025-04-16 08:00:00
4,2.779515,2025-04-17 08:00:00
...,...,...
53,14.935564,2025-06-27 19:00:00
54,14.947142,2025-06-30 19:00:00
55,14.834311,2025-07-01 19:00:00
56,14.826351,2025-07-02 19:00:00


In [ ]:
calculate_mape(all_predict.predict, all_predict.offered)

In [ ]:
(np.mean((all_predict.predict-all_predict.offered)**2))**0.5

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mape ：',calculate_mape(_.predict, _.offered))

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mse ：',(np.mean((_.predict-_.offered)**2))**0.5)

In [ ]:
1

In [ ]:
_ = all_predict[all_predict.hour == 11]

In [ ]:
calculate_mape(_.predict, _.offered)

In [ ]:
all_predict

In [ ]:
feature_[['predict','datetime']]

In [ ]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)


In [ ]:
all_predict.to_excel(r'return.xlsx')

In [ ]:
datetime

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data11 = lstm_data[lstm_data.hour == 11]

In [ ]:
data11.groupby('datetime').min().offered.sort_values()

In [ ]:

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

# 创建示例数据
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['offered'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='真实值'  # 图例名称
))
fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['predict'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='预测值'  # 图例名称
))

# 设置坐标轴名称
fig.update_layout(
    title='折线图示例',
    xaxis_title='X轴',
    yaxis_title='Y轴',
    template='plotly_white'
)
fig.show()

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot( all_predict['datetime'], all_predict['offered'] , label='Actual Data')
plt.plot( all_predict['datetime'], all_predict['predict'], label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
now_lstm_data

In [ ]:
now_lstm_data.offered.min()

In [ ]:
_ = lstm_data.groupby(['year','month','day']).offered.sum()
_.sort_values()[:30]

In [ ]:
now_lstm_data[X_columns]

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot(now_lstm_data[['offered']][1000:].to_numpy(), label='Actual Data')
plt.plot(return_y_pred, label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
predict_y =  model.predict( now_lstm_data[X_columns][:1000].to_numpy().reshape(-1, X_train.shape[1], 1))
predict_y = scaler.inverse_transform(np.array(predict_y).reshape(-1, 1))

In [ ]:
calculate_mape(now_lstm_data[['offered']][:1000].to_numpy(), predict_y)

In [ ]:
_ = ((np.abs((now_lstm_data[['offered']][1000:].to_numpy() - return_y_pred)/now_lstm_data[['offered']][1000:].to_numpy())))*100

In [ ]:
_.sort()

In [ ]:
np.argmax(_)

In [ ]:
_[298]

In [ ]:
now_lstm_data[['offered']][1000:].to_numpy()[298]

In [23]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
import datetime
import numpy as np
from datetime import time


data = pd.read_excel('workspace/RS_KSR_WFM_STAT_2025_05_14.xlsx')
#data = pd.read_excel('RS_KSR_WFM_STAT_2025_05_14.xlsx')

pd.set_option('display.max_rows', 100)

data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

data = data[~data .offered.isna()]
data = data[data .offered != 0]
data_=data
month_number='2025-03-15'
data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
# 假设数据有两列：'timestamp' 和 'value'
holiday_data = pd.read_excel(r'workspace/holiday.xlsx')
#holiday_data = pd.read_excel(r'holiday.xlsx')

holiday_data['~is_holiday'] = 0
Holiday_name = ['Christmas Day', 'Columbus Day',
       'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
       'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
       'Veterans Day']
holiday_data['date'] = holiday_data.Date.dt.date
data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
data['date'] = data.datetime.dt.date

data.datetime = data.datetime.dt.floor('T')
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('30T') if x.minute == 29 else x
)
data['datetime'] = data['datetime'].apply(
    lambda x: x.ceil('H') if x.minute == 59 else x
)
data['time'] = data.datetime.dt.time
data = data.sort_values('datetime').reset_index(drop = True)
data = data.sort_values(by='datetime')
data = data[data.fiscalDate<pd.to_datetime('2025-04-15 00:00:00')]

# 获取所有唯一的日期
unique_dates = data['date'].unique()
data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
data = data[data['~is_holiday'] == 1]

holiday_datetime = holiday_data.date.to_numpy()


unique_times = [
time(9, 0), time(9, 30), time(10, 0),
time(10, 30), time(11, 0), time(11, 30),
time(12, 0), time(12, 30), time(13, 0),
time(13, 30), time(14, 0), time(14, 30),
time(15, 0), time(15, 30), time(16, 0),
time(16, 30), time(17, 0), time(17, 30),
time(18, 0), time(18, 30), time(19, 0)]


def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape
all_predict = pd.DataFrame()
all_feature = pd.DataFrame()
for now_day_pre,now_predict_date in enumerate(pd.date_range(start=unique_dates.max()+ pd.offsets.BDay(32), end=unique_dates.max()+ pd.offsets.BDay(34+23), freq='B')):
    start_time = 32+now_day_pre #用前35+n 到35+n+36天的数据预测当前时刻的数据
    end_time = start_time+36 #
    test_long = 0
    test_start = start_time+test_long
    test_end  = end_time+test_long
    add_column = ['actans','actabn','absActHt','absActSa']
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    f_predict = pd.DataFrame()
    
    
    matrix = []

    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)


        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])

        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix)

    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)

    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)

    matrix = matrix.sort_values('datetime').reset_index(drop = True)

    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)

    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))

    fill_na = fill_na.interpolate(method='linear')

    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)

    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)

    matrix_predict = []




    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)


        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)



        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)

        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)


        year_data = time_data.shift(251)


        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)

        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)

        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-1:]
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)

    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])

    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)

    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)

    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)

    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)

    read_data_predict = matrix_predict.iloc[:]


    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))

    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)

    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = now_predict_date

    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns

    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns


    lstm_data = fill_na.dropna(axis = 0)
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])


    #for week_ in [1,2,3,4,5]:

    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]

        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-test_start])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-test_start] , scaled_data

        X_test_1, y_test =now_lstm_data[X_columns][-test_start:] , now_lstm_data[['offered']][-test_start:]

        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))

        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-test_start:]


        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)

        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-test_start])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集

        X_train_2, y_train =now_lstm_data[X_columns2][:-test_start] , scaled_data

        X_test_2, y_test =now_lstm_data[X_columns2][-test_start:] , now_lstm_data[['offered']][-test_start:]

        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)

        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))

        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-test_start:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):


            f_predict = pd.concat([now_predict_1,f_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            future_data['day'] = now_predict_date
            future_data['hour'] = hour
            future_data['minute'] = minute
            all_feature = pd.concat([all_feature,future_data])
        else:
            f_predict = pd.concat([now_predict_2,f_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            future_data['day'] = now_predict_date
            future_data['hour'] = hour
            future_data['minute'] = minute
            all_feature = pd.concat([all_feature,future_data])
    f_predict = pd.merge(f_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    f_predict = f_predict.sort_values('datetime').reset_index(drop = True)
    all_predict = pd.concat([all_predict,f_predict])
    print(all_predict.head(2))
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
all_predict['diff'] = (all_predict.predict - all_predict.offered)
all_predict['hour'] = all_predict.datetime.dt.hour
all_predict.to_excel(r'workspace/RS_KSR_Test_Daily_{}.xlsx'.format(month_number))
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
data_['conversation_start_interval_tmst']=pd.to_datetime(data_['conversation_start_interval_tmst'])
all_feature['datetime']=pd.to_datetime(all_feature['datetime'])
all_feature = pd.merge(all_feature,data_[['offered','conversation_start_interval_tmst']],left_on = ['datetime'],right_on=['conversation_start_interval_tmst'])
all_feature = all_feature.sort_values('datetime').reset_index(drop = True)
all_feature['mape']=np.abs(all_feature['offered']-all_feature['predict'])/(all_feature['offered'])*100
all_feature.to_excel(r'workspace/RS_KSR_Taget_Daily_{}.xlsx'.format(month_number))


/tmp/ipykernel_1389/3908601848.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
/tmp/ipykernel_1389/3908601848.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['date'] = data.datetime.dt.date
/tmp/ipykernel_1389/3908601848.py:49: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')
/tmp/ipykernel_1389/3908601848.py:49: SettingWithC

Epoch 1/20


E0000 00:00:1747317998.769060    1389 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747317998.787318    1389 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/17 ━━━━━━━━━━━━━━━━━━━━ 1s 220ms/step - loss: 0.0477

KeyboardInterrupt: 